In [0]:
# https://keras.io/
!pip install -q keras
import cv2
import numpy as np

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau


In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 64
compression = 0.5
dropout_rate = 0.35

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


170500096/170498071 [==============================] - 10s 0us/step


In [0]:
#resizing the images to 24X24
def resize_data(x):
    x_scaled = np.zeros((x.shape[0], 24, 24, 3))
    for i, img in enumerate(x):
        new_image = cv2.resize(img, dsize=(24, 24), interpolation=cv2.INTER_CUBIC)
        x_scaled[i] = new_image

    return x_scaled

# resize train and  test data
x_train_24 = resize_data(x_train)
x_test_24 = resize_data(x_test)

img_height_24, img_width_24, channel = x_train_24.shape[1],x_train_24.shape[2],x_train_24.shape[3]
print(img_height_24, img_width_24, channel)

24 24 3


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 64, dropout_rate = 0.35):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 64, dropout_rate = 0.35):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
#create netowrk to first train on 24x24 images
num_filter = 64
dropout_rate = 0.35
l = 12
input_24 = Input(shape=(img_height_24, img_width_24, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input_24)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output_24 = output_layer(Last_Block)



In [0]:
model = Model(inputs=[input_24], outputs=[output_24])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 24, 24, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 24, 24, 64)   1728        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 24, 24, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 24, 24, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=3, min_lr=0.001, mode='auto', min_delta=0.0001)

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
#trying to fit a model for small image size
model.fit(x_train_24, y_train,
                    batch_size=64,
                    epochs=15,
                    verbose=1,
                    callbacks=[reduce_lr],
                    validation_data=(x_test_24, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/15
43776/50000 [=========================>....] - ETA: 44s - loss: 1.5784 - acc: 0.412650000/50000 [==============================] - 378s 8ms/step - loss: 1.5416 - acc: 0.4278 - val_loss: 2.0366 - val_acc: 0.4029
Epoch 2/15
26240/50000 [==============>...............] - ETA: 2:41 - loss: 1.1735 - acc: 0.576050000/50000 [==============================] - 364s 7ms/step - loss: 1.1213 - acc: 0.5959 - val_loss: 1.4098 - val_acc: 0.5798
Epoch 3/15
19584/50000 [==========>...................] - ETA: 3:27 - loss: 0.9722 - acc: 0.654650000/50000 [==============================] - 364s 7ms/step - loss: 0.9447 - acc: 0.6638 - val_loss: 1.4956 - val_acc: 0.5720
Epoch 4/15
17024/50000 [=========>....................] - ETA: 3:45 - loss: 0.8400 - acc: 0.706950000/50000 [==============================] - 364s 7ms/step - loss: 0.8240 - acc: 0.7101 - val_loss: 1.1136 - val_acc: 0.6646
Epoch 5/15
16064/50000 [========>.....................] - E

In [0]:
# Test the model
score = model.evaluate(x_test_24, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [0]:
#removing the last two layers so that we can modify the model to accomodate original images
model.layers.pop()
model.layers.pop()

In [0]:
#save the weights from the model above
model.save_weights("DNST_model_24.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
num_filter = 64
dropout_rate = 0.35
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
BatchNorm = BatchNormalization()(Last_Block)
relu = Activation('relu')(BatchNorm)
AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)



In [0]:
model_32 = Model(inputs=[input], outputs=[AvgPooling])
model_32.summary()

# determine Loss function and Optimizer
model_32.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 32, 32, 64)   1728        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 32, 32, 64)   256         conv2d_53[0][0]                  
__________________________________________________________________________________________________
activation_53 (Activation)      (None, 32, 32, 64)   0           batch_normalization_53[0][0]     
__________________________________________________________________________________________________
conv2d_54 

In [0]:
model_32.load_weights('DNST_model_24.h5')
#model.summary()

In [0]:
flat = Flatten()(model_32.output)
output = Dense(num_classes, activation='softmax')(flat)


In [0]:
model_32=Model(inputs=[model_32.input], outputs=[output])


In [0]:
model_32.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
             metrics=['accuracy'])
model_32.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 32, 32, 64)   1728        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 32, 32, 64)   256         conv2d_53[0][0]                  
__________________________________________________________________________________________________
activation_53 (Activation)      (None, 32, 32, 64)   0           batch_normalization_53[0][0]     
__________________________________________________________________________________________________
conv2d_54 

In [0]:
#creating checkpoints to save the weights after every 10 epochs
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, period=10)
callbacks_list = [checkpoint]

In [0]:
#Load the weights from above and then run the model on train data of actual size, for remaining nmber of epochs
model_32.fit(x_train, y_train,
                    batch_size=32,
                    epochs=35,
                    verbose=1,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/35
21888/50000 [============>.................] - ETA: 5:49 - loss: 0.4974 - acc: 0.829450000/50000 [==============================] - 664s 13ms/step - loss: 0.4849 - acc: 0.8361 - val_loss: 0.6288 - val_acc: 0.8235
Epoch 2/35
50000/50000 [==============================] - 659s 13ms/step - loss: 0.4395 - acc: 0.8501 - val_loss: 0.8203 - val_acc: 0.7554.] - ETA: 0s - loss: 0.4395 - acc: 0.850
Epoch 3/35
50000/50000 [==============================] - 658s 13ms/step - loss: 0.4090 - acc: 0.8584 - val_loss: 0.6493 - val_acc: 0.8140.] - ETA: 0s - loss: 0.4090 - acc: 0.858
Epoch 4/35
50000/50000 [==============================] - 658s 13ms/step - loss: 0.3833 - acc: 0.8683 - val_loss: 0.5505 - val_acc: 0.8438.] - ETA: 0s - loss: 0.3832 - acc: 0.868
Epoch 5/35
50000/50000 [==============================] - 659s 13ms/step - loss: 0.3557 - acc: 0.8769 - val_loss: 0.7073 - val_acc: 0.7933.] - ETA: 0s - loss: 0.3557 - acc: 0.876
Epoch 6/35

In [0]:
# Test the model
score = model_32.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 44s 4ms/step
Test loss: 0.503186296863854
Test accuracy: 0.8749


In [0]:
# Save the trained weights in to .h5 format
model_32.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')